In [12]:
import pandas as pd

from dotenv import load_dotenv
load_dotenv()
import os


import numpy as np
np.set_printoptions(suppress=True)

import datetime
from datetime import date
import mibian

from pandas.io.json import json_normalize


from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
pd.options.plotting.backend = "plotly"

import warnings
warnings.filterwarnings('ignore')

##GVOL API ENDPOINTS 
from gvol import GVol
GVOL_API_KEY = os.getenv('GVOL_API_KEY') #VARIABLE CONTAINED IN .ENV FILE
gvol_client = GVol(header='x-oracle',gvol_api_key=GVOL_API_KEY)


# Deribit Dealers Gamma Profile

In this jupyter notebook we will calculate the gamma profile of dealers at Deribit using Amberdata endpoints.

Despite the standard calculation of GEX (aka Gamma Exposure) calculated in TradFi and other crypto platforms, Amberdata uses the real positioning of dealers for each instruments.

This is possible thanks to the calculations of the "initiator direction" of each single trade ("gvol direction") that relies on most of 30 heuristics comparing the trade with the orderbook snaptshot in the milliseconds before and after the trade occurs.
(for more on this you can check https://twitter.com/genesisvol/status/1555632200694960128?s=20)

Afterwards the positioning of dealers (net inventory) is continuously updated with every trades on Deribit.

Limitations of this model are quite obvious (e.g. not considering cross-platform hedge from dealers or not considering trades not settled on deribit) but anyway this is a powerful tool, and his relevance will become more important as options will gain more traction in the future (like what happened in equities).

## Select a specific datetime and symbol to analyze

Metrics is available as of November 2022.
Metrics is updated with a lag of 1 hour (UTC time)

In [198]:
date_hour = '2023-04-12 08:00' #format YYYY-MM-DD HH:00
symbol = 'BTC'

## Amberdata endpoint "GammaLevelsExpiration" is used

This retrieves the net inventory of dealers for each strike and expiration.

In [199]:
data_gex = gvol_client.options_gvol_gex(symbol=symbol, date= date_hour)
df_gex = pd.json_normalize(data_gex['GammaLevelsExpiration'])
df_gex['date'] = pd.to_datetime(df_gex['date'], unit='ms')
df_gex['expiration'] = pd.to_datetime(df_gex['expiration'], unit='ms')
df_gex['dte'] = (df_gex['expiration'] - df_gex['date']).dt.days + (df_gex['expiration'] - df_gex['date']).dt.seconds/(60*60*24)

## Amberdata endpoint "HifiVolSurfaceStrikesGreeksHourly" is used

This retrieves the orderbook information at the time of calculation. It is needed for retrieving information about implied volatility and spot price, both used in BSM formulas.

In [200]:
data_hifi = gvol_client.options_greeks_hour(exchange='deribit', date=date_hour[:10], symbol=symbol, interval='1 hour')
df_hifi = pd.json_normalize(data_hifi['HifiVolSurfaceStrikesGreeksHourly'])
df_hifi['expiration'] = pd.to_datetime(df_hifi['expiration'], unit='ms')
df_hifi['date'] = pd.to_datetime(df_hifi['date'], unit='ms')

df_hifi = df_hifi[df_hifi['date']== date_hour]

df_hifi.loc[(df_hifi['putCall'] == 'C') & (df_hifi['underlyingPrice'] <= df_hifi['strike']), 'is_otm'] = 'otm'
df_hifi.loc[(df_hifi['putCall'] == 'P') & (df_hifi['underlyingPrice'] >= df_hifi['strike']), 'is_otm'] = 'otm'

df_hifi_otm = df_hifi[(df_hifi['is_otm']=='otm')]

df_final = df_gex.merge(df_hifi_otm[['expiration','strike','markIv','spot']], on =['expiration','strike'])

## BSM gamma computation 

What we want to know is an approximate gamma exposure if the market drops by 10%, or how about if it rallies by 10%. Basically understanding if dealers will be short or long gamma, and at what point they would flip.

To do this we must compute the gamma exposure not just for the current spot but across the "prices" levels.

Using a Black-Scholes formula (using in this case the convenient "mibian" library), we can calculate its gamma since we have most of the ingredients we need from the previous endpoints.

Gamma is calculated for each instruments simulating different price levels, and then using the "dealer net inventory" and a valorized movement of a 1% of spot.

In [201]:
if symbol == 'BTC':
    min_price = max(5000, round((df_final['spot'][0] - 10000)/10000) * 10000)
    max_price = round((df_final['spot'][0] + 10000)/10000) * 10000
    prices = np.arange(min_price, max_price, 250)
elif symbol == 'ETH':
    min_price = max(500, round((df_final['spot'][0] - 1000)/1000) * 1000)
    max_price = round((df_final['spot'][0] + 1000)/1000) * 1000
    prices = np.arange(min_price, max_price, 25)

gamma_chart = []

for exp in df_final['expiration'].unique():
    df_final_exp = df_final[df_final['expiration'] == exp]
    for p in prices:
        df_final_exp['gamma'] = df_final_exp.apply(lambda x: mibian.BS([p, x.strike, 0, x.dte], volatility = x.markIv).gamma, axis=1)
        gamma_sum = (df_final_exp['gamma'] * df_final_exp['dealerNetInventory'] * p * p * 0.01).sum()
        
        gamma_chart.append([exp, p, gamma_sum])

gamma_profile = pd.DataFrame(gamma_chart, columns = ['expiration','spot', 'usd_gamma']).set_index('spot')

## Charting total gamma exposure and each expirations 

In [202]:
##Here you can exclude some expirations
gamma_profile_exp = gamma_profile[gamma_profile['expiration']>'2022-4-12 08:00']
gamma_profile_tot = gamma_profile_exp.groupby(gamma_profile_exp.index).sum()

##Total gamma chart

fig = px.bar(gamma_profile_tot, x = gamma_profile_tot.index, y = 'usd_gamma', color=gamma_profile_tot["usd_gamma"] > 0,
    color_discrete_map={True: "green", False: "red"}, opacity = 0.8)

fig.update_layout(showlegend = False,
                  title = symbol + ' TOTAL Dealers Gamma Profile at Deribit'
                 )
fig.update_yaxes(title='USD Dealers Gamma (1% move)')

fig.add_vline(x=df_final['spot'].mean(), line_width=2, line_dash="dot", line_color="grey")
fig.show()


##Each expirations charts

for exp in gamma_profile['expiration'].unique():
    gamma_profile_exp = gamma_profile[gamma_profile['expiration'] == exp]
    fig = px.bar(gamma_profile_exp, x = gamma_profile_exp.index, y = 'usd_gamma', color=gamma_profile_exp["usd_gamma"] > 0,
        color_discrete_map={True: "green", False: "red"}, opacity = 0.8)

    fig.update_layout(showlegend = False,
                      title = str(exp)[0:10] +' '+symbol+' Dealers Gamma Profile at Deribit'
                     )
    fig.update_yaxes(title='USD Dealers Gamma (1% move)')
    fig.add_vline(x=df_final['spot'].mean(), line_width=2, line_dash="dot", line_color="grey")
    fig.show()